In [2]:
import boto3
import pickle
import h5py
import tables
import numpy as np
import pandas as pd
import sklearn
import caserec

In [4]:
df_game_info=pd.read_pickle('~/Downloads/np_games_info.pkl')

In [6]:
df_game=pd.DataFrame(df_game_info)

In [7]:
df_game.head()

,0,1,2,3,4,5,6,7
0,1,Die Macher,1986,"[3–5, 240, 14+, 4.34]","[[Karl-Heinz Schmiel], [Marcus Gschwendtner, H...",[Strategy],"[Economic, Negotiation, Political]","{'Lil Red Head': {'score': 7, 'stat': ['own'],..."
1,2,Dragonmaster,1981,"[3–4, 30, 12+, 1.96]","[[G. W. ""Jerry"" D'Arcey], [Bob Pepper], [E.S. ...",[Strategy],"[Card Game, Fantasy]","{'ainataqe': {'score': 0, 'stat': ['own'], 'ti..."
2,3,Samurai,1998,"[2–4, 30–60, 10+, 2.50]","[[Reiner Knizia], [Franz Vohwinkel], [Fantasy ...",[Strategy],"[Abstract Strategy, Medieval]","{'oneiric': {'score': 10, 'stat': ['own'], 'ti..."
3,4,Tal der Könige,1992,"[2–4, 60, 12+, 2.67]","[[Christian Beierer], [Thomas di Paolo], [KOSM...",[None],[Ancient],"{'Ewu123': {'score': 0, 'stat': ['own'], 'time..."
4,5,Acquire,1964,"[2–6, 90, 12+, 2.51]","[[Sid Sackson], [Scott Okumura, Peter Whitley]...",[Strategy],[Economic],"{'mikecommons': {'score': 7, 'stat': ['own'], ..."


In [8]:
df_game.columns=['game_id','game_title','release_year','persons_duration_age_rating','publisher','genre','label','user_rating']

In [9]:
df_game.head()

,game_id,game_title,release_year,persons_duration_age_rating,publisher,genre,label,user_rating
0,1,Die Macher,1986,"[3–5, 240, 14+, 4.34]","[[Karl-Heinz Schmiel], [Marcus Gschwendtner, H...",[Strategy],"[Economic, Negotiation, Political]","{'Lil Red Head': {'score': 7, 'stat': ['own'],..."
1,2,Dragonmaster,1981,"[3–4, 30, 12+, 1.96]","[[G. W. ""Jerry"" D'Arcey], [Bob Pepper], [E.S. ...",[Strategy],"[Card Game, Fantasy]","{'ainataqe': {'score': 0, 'stat': ['own'], 'ti..."
2,3,Samurai,1998,"[2–4, 30–60, 10+, 2.50]","[[Reiner Knizia], [Franz Vohwinkel], [Fantasy ...",[Strategy],"[Abstract Strategy, Medieval]","{'oneiric': {'score': 10, 'stat': ['own'], 'ti..."
3,4,Tal der Könige,1992,"[2–4, 60, 12+, 2.67]","[[Christian Beierer], [Thomas di Paolo], [KOSM...",[None],[Ancient],"{'Ewu123': {'score': 0, 'stat': ['own'], 'time..."
4,5,Acquire,1964,"[2–6, 90, 12+, 2.51]","[[Sid Sackson], [Scott Okumura, Peter Whitley]...",[Strategy],[Economic],"{'mikecommons': {'score': 7, 'stat': ['own'], ..."


In [10]:
df_game.shape

(108571, 8)

In [14]:
df_game_only=df_game.drop('user_rating',axis=1)

In [15]:
df_game_only.head()

,game_id,game_title,release_year,persons_duration_age_rating,publisher,genre,label
0,1,Die Macher,1986,"[3–5, 240, 14+, 4.34]","[[Karl-Heinz Schmiel], [Marcus Gschwendtner, H...",[Strategy],"[Economic, Negotiation, Political]"
1,2,Dragonmaster,1981,"[3–4, 30, 12+, 1.96]","[[G. W. ""Jerry"" D'Arcey], [Bob Pepper], [E.S. ...",[Strategy],"[Card Game, Fantasy]"
2,3,Samurai,1998,"[2–4, 30–60, 10+, 2.50]","[[Reiner Knizia], [Franz Vohwinkel], [Fantasy ...",[Strategy],"[Abstract Strategy, Medieval]"
3,4,Tal der Könige,1992,"[2–4, 60, 12+, 2.67]","[[Christian Beierer], [Thomas di Paolo], [KOSM...",[None],[Ancient]
4,5,Acquire,1964,"[2–6, 90, 12+, 2.51]","[[Sid Sackson], [Scott Okumura, Peter Whitley]...",[Strategy],[Economic]


In [16]:
df_game_only.to_csv('game_info_full.csv')

In [31]:
df_extract=pd.read_csv('traning_data_3column_2000.csv')

df_extract=df_extract[['game_id','user_id','rating']]

In [3]:
df_extract.head()

,game_id,user_id,rating
0,0,Lil Red Head,7.0
1,0,hsjx945,8.0
2,0,RuffRyder,0.0
3,0,SkywalterDBZ,7.0
4,0,Olmestig,8.0


In [4]:
df_extract.shape[0]

6792368

In [5]:
df_extract.groupby(['game_id'])['rating'].mean().describe()

count    4238.000000
mean        1.711877
std         0.775608
min         0.000000
25%         1.166667
50%         1.654272
75%         2.196028
max         5.480711
Name: rating, dtype: float64

In [6]:
#df_extract[df_extract.groupby(['game_id'])['rating'].mean()>1.2]
df_filter=df_extract.groupby('game_id').filter(lambda s: s.rating.mean()>=2.2)

In [7]:
df_filter.game_id.nunique()

1052

In [8]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD


# Creation of the dataframe. Column names are irrelevant.
#ratings_dict = {'itemID': [1, 1, 1, 2, 2],
#                'userID': [9, 32, 2, 45, 'user_foo'],
#                'rating': [3, 2, 4, 3, 1]}
df_final = pd.DataFrame(df_filter)

df_final=df_final[['user_id','game_id','rating']]
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_final[['user_id', 'game_id', 'rating']], reader)



In [6]:
algo = SVD()

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

KeyboardInterrupt: 

In [9]:
from surprise import accuracy
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.35)
#trainset_1,testset_1 = train_test_split(trainset, test_size=.25)



In [10]:
# We'll use the famous SVD algorithm.
algo = SVD(n_epochs = 20, n_factors = 50, verbose = True)

predictions = algo.fit(trainset).test(testset)

accuracy.rmse(predictions)
accuracy.mae(predictions)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 2.8251
MAE:  2.1603


2.1602615789314066

In [ ]:
#too slow, kernel crashes
from surprise import KNNBasic
algo_2 = KNNBasic(k=10)
predictions = algo_2.fit(trainset).test(testset)

accuracy.rmse(predictions)
accuracy.mae(predictions)


In [ ]:
### below uses caserecommender #####

In [11]:
df_final['user_id']

0            Lil Red Head
1                 hsjx945
2               RuffRyder
3            SkywalterDBZ
4                Olmestig
                ...      
6792363        bigbadbobo
6792364    therailbaron85
6792365       BoardGameCo
6792366      speedowagon1
6792367            animal
Name: user_id, Length: 5237198, dtype: object

In [12]:
df_final.head()
codes, uniques=pd.factorize(df_final['user_id'])

In [13]:
codes

array([     0,      1,      2, ...,  11853,  12519, 192798])

In [14]:
df_final['user_num_id']=codes

In [15]:
df_final.head()
df_final_numeric=df_final[['user_num_id','game_id','rating']]

In [16]:
from caserec.recommenders.rating_prediction.itemknn import ItemKNN
from sklearn.model_selection import train_test_split



train_set, test_set = train_test_split(df_final_numeric, test_size=.25)

train_set.to_csv('train_set.csv',index=False,header=None)
test_set.to_csv('test_set.csv',index=False,header=None)



In [ ]:
ItemKNN('train_set.csv', 'test_set.csv',sep=',').compute()

[Case Recommender: Rating Prediction > ItemKNN Algorithm]

train data:: 338965 users and 1052 items (3927898 interactions) | sparsity:: 98.90%
test data:: 257104 users and 1049 items (1309300 interactions) | sparsity:: 99.51%



In [1]:
#get top n recommendation for a particular user

from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=5):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
print(trainset)
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

NameError: name 'data' is not defined